In [1]:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

# Delta tables

Spark SQL has become a standard way to process data in a distributed way. It's dataframe and dataset api allow us to do all kinds of transformations with our data. Normaly, when we speak about file formats in big data, we mostly use parquet, a columnar file format created for big data precesess. The amount of metadata, and columnar way to storage it, allows us to try to read only the esencial data, but for this format, and for all formats in big data, we have a problem to administrate this files in a file system point of view. When we need to do a mantainance in this tables, due to cleaning, or GDPR actions, we allways work more or less in the following way.
First, stop everithing related with this file, nobody can read it.
Second, launch the process and write the result in other folder.

In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`io.delta::delta-core:0.5.0`
import $ivy.`io.circe::circe-parser:0.12.3`
import $ivy.`com.lihaoyi::os-lib:0.2.7`
import org.apache.spark.sql._
import io.circe.parser._
import java.time.Instant

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.5/spark-sql_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.9.1/almond-spark_2.12-0.9.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.5/spark-parent_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.5/spark-tags_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.6.7.3/jackson-databind-2.6.7.3.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammonite-spark_2.12/0.7.2/ammonite-spark_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sketch_2.12/2.4.5/spark-sketch_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.5/spark-catalyst_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/mave

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-launcher_2.12/2.4.5/spark-launcher_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/javax/activation/activation/1.1.1/activation-1.1.1.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-graphite/3.1.5/metrics-graphite-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/com/clearspring/analytics/stream/2.7.0/stream-2.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/lz4/lz4-java/1.4.0/lz4-java-1.4.0.pom
Downloaded https://repo1.maven.org/maven2/oro/oro/2.0.8/oro-2.0.8.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-memory/0.10.0/arrow-memory-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/xerial/snappy/snappy-java/1.1.7.3/snappy-java-1.1.7.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.5/spark-unsafe_2.12-2.4.5.pom
Downloaded https

Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-format/0.10.0/arrow-format-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.20.v20190813/jetty-server-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-core/2.7.9/jackson-core-2.7.9.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.6.7.1/jackson-module-scala_2.12-2.6.7.1.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet/2.22.2/jersey-container-servlet-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/3.1.5/metrics-core-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/janino/3.0.9/janino-3.0.9.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-format/2.4.0/p

Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/41/commons-parent-41.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.7/jackson-parent-2.7.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.6.1/jackson-parent-2.6.1.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.16/slf4j-parent-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/40/commons-parent-40.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/35/commons-parent-35.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/34/commons-parent-34.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/23/commons-parent-23.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/17/commons-parent-17.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/17/apache-17.pom
Downl

Downloaded https://repo1.maven.org/maven2/org/fusesource/leveldbjni/leveldbjni-all/1.8/leveldbjni-all-1.8.pom
Downloaded https://repo1.maven.org/maven2/com/esotericsoftware/kryo-shaded/4.0.2/kryo-shaded-4.0.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-hdfs/2.6.5/hadoop-hdfs-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/hk2-api/2.4.0-b34/hk2-api-2.4.0-b34.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-app/2.6.5/hadoop-mapreduce-client-app-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-jobclient/2.6.5/hadoop-mapreduce-client-jobclient-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/tukaani/xz/1.5/xz-1.5.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/media/jersey-media-jaxb/2.22.2/jersey-media-jaxb-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.1/scala-reflect-2.12.1.pom
Downloaded ht

...

Downloaded https://repo1.maven.org/maven2/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final-sources.jar
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-json/3.1.5/metrics-json-3.1.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/javassist/javassist/3.18.1-GA/javassist-3.18.1-GA-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet/2.22.2/jersey-container-servlet-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/media/jersey-media-jaxb/2.22.2/jersey-media-jaxb-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/core/jersey-client/2.22.2/jersey-client-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-graphite/3.1.5/metrics-graphite-3.1.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/core/jersey-common/2.22.2/jersey-common-2.22.2-sources.jar
Downlo

Downloaded https://repo1.maven.org/maven2/com/vlkan/flatbuffers/1.2.0-3f79e055/flatbuffers-1.2.0-3f79e055-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet-core/2.22.2/jersey-container-servlet-core-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-format/0.10.0/arrow-format-0.10.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-encoding/1.10.1/parquet-encoding-1.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.5/spark-catalyst_2.12-2.4.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-io/9.4.20.v20190813/jetty-io-9.4.20.v20190813-sources.jar
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-http/9.4.20.v20190813/jetty-http-9.4.20.v20190813-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-vector/0.10.0/arrow-vector-0.10.0-sources.jar
Down

Downloaded https://repo1.maven.org/maven2/org/antlr/antlr-runtime/3.5.2/antlr-runtime-3.5.2.pom
Downloaded https://repo1.maven.org/maven2/com/ibm/icu/icu4j/58.2/icu4j-58.2.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/ST4/4.0.8/ST4-4.0.8.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/javax.json/1.0.4/javax.json-1.0.4.pom
Downloaded https://repo1.maven.org/maven2/org/abego/treelayout/org.abego.treelayout.core/1.0.3/org.abego.treelayout.core-1.0.3.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr-master/3.5.2/antlr-master-3.5.2.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/json/1.0.4/json-1.0.4.pom
Downloaded https://repo1.maven.org/maven2/org/abego/treelayout/org.abego.treelayout.core/1.0.3/org.abego.treelayout.core-1.0.3.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/javax.json/1.0.4/javax.json-1.0.4.jar
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr-runtime/3.5.2/antlr-runtime-3.5.2.jar
Downloaded https://repo1.m

Downloaded https://repo1.maven.org/maven2/com/lihaoyi/geny_2.12/0.1.5/geny_2.12-0.1.5.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/geny_2.12/0.1.5/geny_2.12-0.1.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.2.7/os-lib_2.12-0.2.7-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.2.7/os-lib_2.12-0.2.7.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.7/scala-library-2.12.7-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.7/scala-library-2.12.7.jar


import $ivy.$                                  

import $ivy.$                           

import $ivy.$                              

import $ivy.$                          

import org.apache.spark.sql._

import io.circe.parser._

In [3]:
val file = "delta-example"

file: String = "delta-example"

In [4]:
val wd = os.pwd / file

wd: os.package.pwd.ThisType = root/'home/'jovyan/'work/"delta-table-first-steps"/"delta-example"

In [5]:
os.remove.all(wd)

In [6]:
val spark = SparkSession.builder().appName("delta-test").master("local[1]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/03/20 11:21:47 INFO SparkContext: Running Spark version 2.4.5
20/03/20 11:21:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/03/20 11:21:48 INFO SparkContext: Submitted application: delta-test
20/03/20 11:21:48 INFO SecurityManager: Changing view acls to: jovyan
20/03/20 11:21:48 INFO SecurityManager: Changing modify acls to: jovyan
20/03/20 11:21:48 INFO SecurityManager: Changing view acls groups to: 
20/03/20 11:21:48 INFO SecurityManager: Changing modify acls groups to: 
20/03/20 11:21:48 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/03/20 11:21:48 INFO Utils: Successfully started service 'sparkDriver' on port 34939.
20/0

spark: SparkSession = org.apache.spark.sql.SparkSession@421a47af

In [7]:
spark.sparkContext.setLogLevel("ERROR")

# Version 0

Now that we have our spark session, we want to try this delta format, so we are going to write some data to it. The only diference to write parquet, is that the format is "delta".

In [8]:
spark.range(1,10).write.format("delta").save(file)

To be sure, lets show what we have written.

In [9]:
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



Take a look of the folder were we wrote. We will se that we have written a single parquet file, due to the single core of our standalone cluster. But also, it created a folder called `_delta_log`

In [10]:
os.list(wd).foreach(println)

/home/jovyan/work/delta-table-first-steps/delta-example/.part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet.crc
/home/jovyan/work/delta-table-first-steps/delta-example/_delta_log
/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet


Listing the files, we see that it contais a single json file.

In [11]:
val logFolder = wd / "_delta_log"
os.list(logFolder).foreach(println)

/home/jovyan/work/delta-table-first-steps/delta-example/_delta_log/00000000000000000000.json


logFolder: wd.ThisType = root/'home/'jovyan/'work/"delta-table-first-steps"/"delta-example"/'_delta_log

We can use [circe](https://circe.github.io/circe/) to see the content of this file in a pretty way.

In [12]:
val version0 = os.list(logFolder).find(_.last.endsWith("0.json")).get
val version0Json = os.read.lines(version0).map(x => parse(x).right.get.spaces2)

version0: os.Path = root/'home/'jovyan/'work/"delta-table-first-steps"/"delta-example"/'_delta_log/"00000000000000000000.json"
version0Json: collection.mutable.WrappedArray[String] = WrappedArray(
  """{
  "commitInfo" : {
    "timestamp" : 1584703320911,
...

Each line of the file is a json document.The first one, has the commit infomation, showing us the timestamp of the commit, the type of operation, with the parameters of it, and also if it was a bling append, an append that didn't take previous commits in account.

In [13]:
println(version0Json(0))

{
  "commitInfo" : {
    "timestamp" : 1584703320911,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "ErrorIfExists",
      "partitionBy" : "[]"
    },
    "isBlindAppend" : true
  }
}


The second document, contains information of the protocol used for delta table, with the minimun version of reader and writer needed.

In [14]:
println(version0Json(1))

{
  "protocol" : {
    "minReaderVersion" : 1,
    "minWriterVersion" : 2
  }
}


The third, contains metadata information, with the schema, the format of the files used, in this case parquet. 

In [15]:
println(version0Json(2))

{
  "metaData" : {
    "id" : "263a8252-897c-4e17-984e-2e492d649a6b",
    "format" : {
      "provider" : "parquet",
      "options" : {
        
      }
    },
    "schemaString" : "{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns" : [
    ],
    "configuration" : {
      
    },
    "createdTime" : 1584703319985
  }
}


And the last, shows that it has added a new file with the path of the new parquet file created.

In [16]:
println(version0Json(3))

{
  "add" : {
    "path" : "part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 483,
    "modificationTime" : 1584703320000,
    "dataChange" : true
  }
}


Let's append a new bunch of data, and see how has changed the folder. And look what contains in it now.

In [17]:
spark.range(10,20).write.format("delta").mode(SaveMode.Append).save(file)
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



Nothing unexpected, it mixed the previous data with the new one. If we check the parquet files, we will haver our previous file, and a new one, as expected.

In [18]:
os.list(wd).filter(_.last.endsWith("parquet")).foreach(println)

/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-3bbea36c-9cd7-430e-be6b-5bb9ae30770a-c000.snappy.parquet
/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet


And in the log? Take a look.

In [19]:
os.list(logFolder).foreach(println)

/home/jovyan/work/delta-table-first-steps/delta-example/_delta_log/00000000000000000000.json
/home/jovyan/work/delta-table-first-steps/delta-example/_delta_log/00000000000000000001.json


We can see that there is a new file that ends in 0001.json, this is the latest version of the log, let's take a look

In [20]:
val version1 = os.list(logFolder).find(_.last.endsWith("1.json")).get
val version1Json = os.read.lines(version1).map(x => parse(x).right.get)

version1: os.Path = root/'home/'jovyan/'work/"delta-table-first-steps"/"delta-example"/'_delta_log/"00000000000000000001.json"
version1Json: collection.mutable.WrappedArray[io.circe.Json] = WrappedArray(
  JObject(
    object[commitInfo -> {
  "timestamp" : 1584703330351,
...

This `version 1` file only has two lines, one like the previous one with the information of the commit.

In [21]:
println(version1Json(0))

{
  "commitInfo" : {
    "timestamp" : 1584703330351,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "Append",
      "partitionBy" : "[]"
    },
    "readVersion" : 0,
    "isBlindAppend" : true
  }
}


And the second one, also like the [version 0](#Version-0)

In [22]:
println(version1Json(1))

{
  "add" : {
    "path" : "part-00000-3bbea36c-9cd7-430e-be6b-5bb9ae30770a-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 472,
    "modificationTime" : 1584703330000,
    "dataChange" : true
  }
}


You won't see any difference between delta table and workning with parquet at this point.

But lets overwrite our data, and we will see the first real difference.

In [23]:
spark.range(20,30).write.format("delta").mode(SaveMode.Overwrite).save(file)

Let's check that the overwrite was done ok.

In [24]:
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
| 20|
| 21|
| 22|
| 23|
| 24|
| 25|
| 26|
| 27|
| 28|
| 29|
+---+



If we list our parquet files, we will see that not only it has added a new one, it also mantains the previous that we created.

In [25]:
os.list(wd).filter(_.last.endsWith("parquet")).foreach(println)

/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-3bbea36c-9cd7-430e-be6b-5bb9ae30770a-c000.snappy.parquet
/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet
/home/jovyan/work/delta-table-first-steps/delta-example/part-00000-ca61a7ed-9ea7-4e13-8085-59268c2d055a-c000.snappy.parquet


Why? The anser is in the new log file created, this time with index 2.

In [26]:
val version2 = os.list(logFolder).find(_.last.endsWith("2.json")).get
val version2Json = os.read.lines(version2).map(x => parse(x).right.get.spaces2)

version2: os.Path = root/'home/'jovyan/'work/"delta-table-first-steps"/"delta-example"/'_delta_log/"00000000000000000002.json"
version2Json: collection.mutable.WrappedArray[String] = WrappedArray(
  """{
  "commitInfo" : {
    "timestamp" : 1584703450193,
...

In [27]:
println(version2Json(0))

{
  "commitInfo" : {
    "timestamp" : 1584703450193,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "Overwrite",
      "partitionBy" : "[]"
    },
    "readVersion" : 1,
    "isBlindAppend" : false
  }
}


In [28]:
println(version2Json(1))
println(version2Json(2))
println(version2Json(3))

{
  "add" : {
    "path" : "part-00000-ca61a7ed-9ea7-4e13-8085-59268c2d055a-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 472,
    "modificationTime" : 1584703449000,
    "dataChange" : true
  }
}
{
  "remove" : {
    "path" : "part-00000-3bbea36c-9cd7-430e-be6b-5bb9ae30770a-c000.snappy.parquet",
    "deletionTimestamp" : 1584703450192,
    "dataChange" : true
  }
}
{
  "remove" : {
    "path" : "part-00000-47effd1c-2417-4499-904b-7d74392e1692-c000.snappy.parquet",
    "deletionTimestamp" : 1584703450193,
    "dataChange" : true
  }
}


Now we see that the log is indicating that its adding a new parquet file (line with "add" key), but also, that in this version, it must not take in account the previous files(lines with the "remove" key).

Now we can uderstand that what delta table is doing. It isn't resolving the actions in the folder like parquet that would have aresed all previous data. What delta is doing it's keeping all the data files, in this case parquet files, and logging the changes in the '_delta_log' throw versioning.

This way, delta can recreate in any moment no only the last version, as we would do in a normal parquet file, also previous state versions.

So if we can recreate them, how we can read the file, but like the version 0 or 1? indicating in the options the version that we want to read.

In [29]:
spark.read.format("delta").option("versionAsOf", 0).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [30]:
spark.read.format("delta").option("versionAsOf", 1).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



Also, we can use the option "timestampAsOf" to get the last version available at that moment. For this we can take advance of the optics implementation in circe JSON object to obtain the timestamp of the version 1.

In [31]:
val version1Timestamp = version1Json(0).hcursor.downField("commitInfo").get[Long]("timestamp").right.get

version1Timestamp: Long = 1584703330351L

And create two timestamps, one slightly earlier to the version 1 and other later to version 1. The format of the date must be 

In [38]:
val previousToVersion1 = Instant.ofEpochMilli(version1Timestamp - 5000)
val laterToVersion1 = Instant.ofEpochMilli(version1Timestamp + 5000)

previousToVersion1: Instant = 2020-03-20T11:22:05.351Z
laterToVersion1: Instant = 2020-03-20T11:22:15.351Z

In [39]:
spark.read.format("delta").option("timestampAsOf", previousToVersion1.toString).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [40]:
spark.read.format("delta").option("timestampAsOf", laterToVersion1.toString).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+

